<div style="text-align: center;">
<h1>Reel Realities: How Gender and Age Shape Success Across Box Office and Streaming Platforms</h1>
</div>

### <u>Imports</u>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from src.data.dataloader import (
    CMUDatasetLoader, IMDBDatasetLoader, KaggleDatasetLoader, NumbersDatasetLoader)
import src.utils.data_utils as data_utils
import src.utils.evaluation_utils as evaluation_utils
from src.utils.EDA_utils import EDA
from src.scripts.streaming import get_streaming_dataframe
import warnings

# Suppress specific FutureWarnings about is_sparse being deprecated
warnings.filterwarnings(action="ignore", category=FutureWarning, module="sklearn", message=".*is_sparse is deprecated.*")

### 1. <u>Data cleaning and pre-processing</u>

#### 1.1 CMU Dataset

We will use three files of the CMU dataset:
1. "plot_summaries.txt" gives us the plots of the movies.
2. "movie.metadata.tsv" gives us information about the languages, countries, and genres of the movies.
3. "character.metadata.tsv" gives us information about actors and the characters they play.

In [ ]:
directory = "data"
CMU_loader = CMUDatasetLoader(directory)
plot_summaries_df = CMU_loader.load_plot_summaries()

print(f"The plot summaries dataframe has {len(plot_summaries_df):,} values.")

In [ ]:
null_id = plot_summaries_df["Wikipedia_movie_ID"].isnull().sum()
empty_id = plot_summaries_df[plot_summaries_df["Wikipedia_movie_ID"] == ""]["Wikipedia_movie_ID"].sum()
null_plot = plot_summaries_df["Plot Summaries"].isnull().sum()
empty_plot = plot_summaries_df[plot_summaries_df["Plot Summaries"] == ""]["Plot Summaries"].sum()

print(f"Number of null values in 'Wikipedia_movie_ID': {null_id}")
print(f"Number of empty values in 'Wikipedia_movie_ID': {empty_id}")
print(f"Number of null values in 'Plot Summaries': {null_plot}")
print(f"Number of empty values in 'Plot Summaries': {empty_plot}")

In [ ]:
movie_metadata_df = CMU_loader.load_movie_metadata()
print(f"The movie metadata dataframe has {len(movie_metadata_df):,} values.")

In [5]:
movie_metadata_df["Movie_name"] = (movie_metadata_df["Movie_name"]
                    .str.strip()  #Remove leading/trailing whitespace
                    .str.lower()  #Convert to lowercase
                    .replace("", np.nan)  #Replace empty strings with NaN
                   )

In [6]:
# Movie languages, countries, and genres are formatted as a dictionary, but they are actually a string.
# We convert them to comma seperated strings. 

column_names_to_clean = ["Movie_countries", "Movie_languages", "Movie_genres"]
movie_metadata_df = data_utils.convert_dicts_to_strings(movie_metadata_df, column_names_to_clean)

# Cleaning the strings
movie_metadata_df = data_utils.clean_string_columns(movie_metadata_df, column_names_to_clean)

In [ ]:
# There are inconsistencies in the date format: some rows have the date as year only, others as year-month, and some as year-month-day. 
# We convert them all to the year format only since the month and day are not relevant to our analysis.
movie_metadata_df, date_pattern_summary = data_utils.standardize_dates(movie_metadata_df, "Movie_release_date")
print("Pattern Summary:\n", date_pattern_summary)

In [ ]:
# Looking at the percentage of null values in the dataset
n_null_movie_metadata = ((movie_metadata_df.isnull().sum() / len(movie_metadata_df)) * 100).apply(lambda x: f"{x:,.2f}%")

print(
    f"Percentage of null values per column:\n"
    f"movie_metadata_df:\n{n_null_movie_metadata}"
)

In [9]:
character_metadata_df = CMU_loader.load_character_metadata()

In [10]:
character_metadata_df["Actor_name"]= character_metadata_df["Actor_name"].fillna("unknown").str.lower()

# Ensures that the date columns are correctly interpreted as datetime objects in pandas. 
character_metadata_df["Movie_release_date"] = pd.to_datetime(character_metadata_df["Movie_release_date"], errors="coerce").dt.date
character_metadata_df["Actor_date_of_birth"] = pd.to_datetime(character_metadata_df["Actor_date_of_birth"], errors="coerce").dt.date

# Drop rows where "Movie_release_date" or "Actor_date_of_birth" is NaT
character_metadata_df.dropna(subset=["Movie_release_date", "Actor_date_of_birth"], inplace=True)

# Calculate Actor_age_at_movie_release if missing and data is available
character_metadata_df["Actor_age_at_movie_release"] = character_metadata_df.apply(
    lambda row: (row["Movie_release_date"] - row["Actor_date_of_birth"]).days // 365 if pd.isnull(row["Actor_age_at_movie_release"]) else row["Actor_age_at_movie_release"],
    axis=1
)

In [ ]:
print(f"Movies without actor gender data: {character_metadata_df['Actor_gender'].isnull().sum()}") 

We therefore drop the movies with no information on the gender of the actors.

In [12]:
character_metadata_df.dropna(subset=["Actor_gender"], inplace=True)

Now, let's group the actors together by creating a *Cast* column on the *movie_metadata_df*. 

In [13]:
cast_per_movie = character_metadata_df.groupby("Freebase_movie_ID")["Actor_name"].apply(lambda x: ", ".join(x)).reset_index()
cast_per_movie = cast_per_movie.rename(columns={"Actor_name": "Cast"})

movie_metadata_df = movie_metadata_df.merge(cast_per_movie, on="Freebase_movie_ID", how="left")

In [ ]:
missing_cast_count = movie_metadata_df["Cast"].isna().sum()
total_movies = len(movie_metadata_df)
print(f"Movies without actor data: {missing_cast_count} out of {total_movies} ({missing_cast_count / total_movies * 100:.2f}%)")

We decide to drop the movies without actor data since it is one of the most important features of our study. 

In [15]:
movie_metadata_df = movie_metadata_df.dropna(subset = ["Cast"])

In [16]:
# Create columns for male, female actor counts
actor_counts = character_metadata_df.groupby("Freebase_movie_ID")["Actor_gender"].value_counts().unstack(fill_value=0)
actor_counts = actor_counts.rename(columns={"M": "Male_actors", "F": "Female_actors"})

# Ensure all columns are present (set missing columns to 0 if necessary)
for col in ["Male_actors", "Female_actors"]:
    if col not in actor_counts.columns:
        actor_counts[col] = 0

actor_counts = actor_counts.reset_index()

# Merge actor counts with the movie_metadata_df DataFrame
movie_metadata_df = movie_metadata_df.merge(actor_counts, on="Freebase_movie_ID", how="left")

# Fill NaN values with 0 
movie_metadata_df[["Male_actors", "Female_actors"]] = movie_metadata_df[["Male_actors", "Female_actors"]].fillna(0)

In [17]:
# Calculate the percentage of female actors in each movie and round to two decimal places
movie_metadata_df["Female_actor_percentage"] = (
    (movie_metadata_df["Female_actors"] / 
     (movie_metadata_df["Female_actors"] + movie_metadata_df["Male_actors"])) * 100
).round(2)

#### 1.2 IMDB Dataset

We will use two IMDB datasets to decribe movies:
1. "title.ratings.tsv" gives us the ratings of the movies as voted by viewers. 
2. "title.basics.tsv", indexes into "title.ratings.tsv" using a alphanumeric unique identifier of the title. It gives general information about the movie such as runtime, release date and adult rating.
3. "title.crew.tsv", indexes into the previous two using the same alphanumeric unique identifier of the title. It gives information on the directors and writers of the movie.

Reference:
Internet Movie Database. (2024). IMDb non-commercial datasets. Retrieved from https://developer.imdb.com/non-commercial-datasets/

In [ ]:
# Loading the IMDB datasets.
IMDB_loader = IMDBDatasetLoader(directory)
IMDB_ratings_df = IMDB_loader.load_ratings()

IMDB_basics_df = IMDB_loader.load_basics()

IMDB_crew_df = IMDB_loader.load_crew()

In [ ]:
# Checking the lengths of the datasets
print(f"Length of IMDB_ratings_df: {len(IMDB_ratings_df):,}\n"
      f"Length of IMDB_basics_df: {len(IMDB_basics_df):,}\n"
      f"Length of IMDB_crew_df: {len(IMDB_crew_df):,}")

Before dealing with the null values we will merge the dataframes together using the alphanumeric unique identifier.

In [20]:
# Merging all three datasets.
IMDB_merged_df = pd.merge(IMDB_ratings_df, IMDB_basics_df, how="inner", left_on="tconst", right_on="tconst")
IMDB_merged_df = pd.merge(IMDB_merged_df, IMDB_crew_df, how="inner", on="tconst")

In [ ]:
print(f"The resulting merged dataframe has length: {len(IMDB_merged_df):,}.")

Formatting the strings appropriately.

In [22]:
IMDB_merged_df = data_utils.clean_string_columns(IMDB_merged_df, ["Original_title", "Primary_title"])

We can now look at null values in the merged IMDB dataframe.

In [ ]:
# Checking the percentage of null values in the dataset.
n_null_IMDB = ((IMDB_merged_df.isnull().sum() / len(IMDB_merged_df)) * 100).apply(lambda x: f"{x:,.2f}%")

print(
    f"Percentage of null values per column:\n"
    f"IMDB_ratings_df:\n{n_null_IMDB}"
)

The end year is always missing. Other than that the proportion of missing values is very small (< 12%). End year is always null for movies and only makes sense for TV shows. It can thus be dropped. We can also drop the titleType column since we know they are all movies after the filtering that was done above.

In [ ]:
# Dropping unnecessary columns.
IMDB_merged_df = IMDB_merged_df.drop(columns=["End_year", "Title_type"], axis=1)

print(f"The resulting dataframe has {len(IMDB_merged_df):,} rows.")

We will use ratings from the IMDB dataset during our study. However, these ratings are based on viewer votes. Initially we thought of discarding rows with too few votes. However, there could be a link between number of votes and number of views of a movie (although definitely not a direct one). We thus decided to keep all rows for the analysis.

#### 1.3 Merging the datasets

##### 1.3.1 Merging IMDB and CMU Movies

In [ ]:
# Merging on the original title.
merge1 = pd.merge(IMDB_merged_df, movie_metadata_df, how="inner", left_on="Original_title", right_on="Movie_name")
# Merging on the primary title.
merge2 = pd.merge(IMDB_merged_df, movie_metadata_df, how="inner", left_on="Primary_title", right_on="Movie_name")

# Concatenating and dropping duplicates that appear from movies with the same Original_title and Primary_title.
movie_df = pd.concat([merge1, merge2]).drop_duplicates(subset=["tconst"]).reset_index(drop=True)

print(f"The resulting dataframe has {len(movie_df):,} rows.")

Some columns appear twice. Let's take a look at the proportion of null values in each duplicate column.

In [ ]:
n_null_movie = (movie_df.isnull().sum()/len(movie_df)).apply(lambda x:f"{x:.2%}")

print(f"Percentage of null values per column:\n{n_null_movie}")

We can see:
- Movie_runtime_x and Movie_runtime_y have 6.02% and 11.58% missing values respectively. We will combine the non null values from both these columns into a new column called runtimeMinutes and then drop the previous two columns. 
- Movie_genres_x has 1.54% missing values against 0.00% missing values for Movie_genres_y. Furthermore Movie_genres_y is from the CMU dataset and seems more complete. We will thus drop the genres_x column.

In [27]:
# Combining all non-null values from Movie_runtime_x and Movie_runtime_y into runtimeMinutes.
movie_df["Movie_runtime"] = movie_df["Movie_runtime_x"].combine_first(movie_df["Movie_runtime_y"])

# Dropping the unnecessary columns.
movie_df.drop(columns=["Movie_runtime_x", "Movie_runtime_y", "Movie_genres_x"], inplace=True)

Let's now see if Movie_name, originalTitle and primaryTitle are all necessary or if there are any redundancies.

In [ ]:
test1 = movie_df["Movie_name"] == movie_df["Primary_title"]  
test2 = movie_df["Movie_name"] == movie_df["Original_title"]

# Checking if there are any movies for which Movie_name is not either in Primary_title or Original_title
print(f"There are {(~(test1 | test2)).sum().item()} movies for which Movie_name is in neither Primary_title ot Original_title.")

We can see the Movie_name column is redundant as its information is either in primaryTitle or in originalTitle. We can thus drop this column.

In [29]:
movie_df.drop(columns="Movie_name", inplace=True)

We can now also drop movie identifier columns Freebase_movie_ID (as everything is already indexed). We will need tconst later on so we keep it.

In [30]:
movie_df.drop(columns=["Freebase_movie_ID"], inplace=True)

To connect this with the plot_summaries_df, we perform a left merge.

In [31]:
movie_df = pd.merge(movie_df, plot_summaries_df, on="Wikipedia_movie_ID", how="left")

Finally, we can drop Wikipedia_movie_ID, which is also a movie identifier column.

In [32]:
movie_df.drop(columns="Wikipedia_movie_ID", inplace=True)

##### 1.3.3 Merging Kaggle and The numbers datasets to the rest

We have a problem, a lot of missing values(NaN or 0) in Movie box office revenue in movie_df and no production budget included. 
These two new datasets include information about movies revenue and budget and other attributes which are crucial for our analysis.

- `Kaggle_df`: Contains important details about movies, including budgets, genres, and revenue.
- `TheNumbers_df`: Provides detailed financial data, focusing on budget and box office gross both domestically and worldwide.

In [33]:
Kaggle_df = KaggleDatasetLoader(directory).load_kaggle()
TheNumbers_df = NumbersDatasetLoader(directory).load_numbers()

Quickly view the structure of the dataframes to understand the data types and confirm data has loaded correctly

In [ ]:
Kaggle_df.head()

In [35]:
# Clean and convert financials to float for consistency with the movie_df
TheNumbers_df = data_utils.clean_currency_columns(TheNumbers_df, ["production_budget", "domestic_gross", "worldwide_gross"])

In [ ]:
TheNumbers_df.head()

We now perform an outer merge to include all records from both datasets, aligning them based on movie titles.


In [37]:
# Normalize the title columns for case insensitive matching
Kaggle_df = data_utils.clean_string_columns(Kaggle_df, ["original_title"])
TheNumbers_df = data_utils.clean_string_columns(TheNumbers_df, ["movie"])

# Merge both dataframes on the title columns
merged_df = pd.merge(Kaggle_df, TheNumbers_df, left_on="original_title", right_on="movie", how="outer")

##### Post-merge cleaning

We create the main financial columns ("movie_budget" and "movie_revenue") by selecting non-null and non-zero values, prioritizing data from TheNumbers dataset when available.


In [38]:
merged_df["movie_budget"] = np.where((merged_df["production_budget"].notna() & (merged_df["production_budget"] != 0)),
                                     merged_df["production_budget"],
                                     merged_df["budget"])

merged_df["movie_revenue"] = np.where((merged_df["worldwide_gross"].notna() & (merged_df["worldwide_gross"] != 0)),
                                      merged_df["worldwide_gross"],
                                      merged_df["revenue"])

# Drop the original columns that were merged into new ones
merged_df.drop(columns=["budget", "revenue", "production_budget", "worldwide_gross", "movie"], inplace=True)

Filter out movies where both budget and revenue data are missing or zero, as they do not provide value for financial analysis.


In [39]:
# Create a mask where both columns are either zero or NaN
mask = (
    (merged_df["movie_budget"].isna() | (merged_df["movie_budget"] == 0)) &
    (merged_df["movie_revenue"].isna() | (merged_df["movie_revenue"] == 0))
)

# Apply the mask and drop those rows
df_filtered = merged_df[~mask]

In [ ]:
print("The number of available and relevant movie budget and revenue information is " + str(df_filtered["movie_budget"].size))

df_filtered.head()

Two merging strategies are employed here to maximize the potential for matching records across the datasets:

Original Title Merge: Merging based on the Original_title from movie_df to the original_title from df_filtered to catch the most direct title matches. 

Primary Title Merge: Using Primary_title as an alternative matching criterion, knowing that some movies might be listed under alternate titles.


In [ ]:
# Normalize the title columns for case-insensitive matching
movie_df["Original_title"] = movie_df["Original_title"].str.strip().str.lower()
movie_df["Primary_title"] = movie_df["Primary_title"].str.strip().str.lower()

# Merging on the original title.
merge1 = pd.merge(movie_df, df_filtered, how="left", left_on="Original_title", right_on="original_title")
# Merging on the secondary title.
merge2 = pd.merge(movie_df, df_filtered, how="left", left_on="Primary_title", right_on="original_title")

# Concatenating and dropping duplicates that appear from movies with the same Original_title and Primary_title.
final_df = pd.concat([merge1, merge2]).drop_duplicates().reset_index(drop=True)

# Filter out movies where revenue data is missing or zero and prioritize initial revenue
final_df["final_movie_revenue"] = np.where(
    (final_df["Movie_box_office_revenue"].notna() & (final_df["Movie_box_office_revenue"] != 0)),
    final_df["Movie_box_office_revenue"],
    final_df["movie_revenue"]
)

final_df.drop(columns=["Movie_box_office_revenue", "movie_revenue", "original_title", "genres", "id", "release_date"], inplace=True, errors="ignore")

# Renaming the columns to follow the convention
rename_columns = {"popularity":"Popularity", 
                  "production_companies":"Production_companies", 
                  "prodcution_countries":"Production_countries",
                  "domestic_gross":"Domestic_gross",
                  "movie_budget":"Movie_budget",
                  "final_movie_revenue":"Final_movie_revenue",
                  "Movie_genres_y":"Movie_genres"}

final_df.rename(columns=rename_columns, inplace=True)


print(f"The resulting dataframe has {len(final_df):,} rows.")
final_df.head()

### 2. Our success metric

#### Financial Analysis

##### Calculating Return on Investment (ROI)

To provide insights into the financial success of the movies, we calculate the Return on Investment (ROI). This metric is derived by comparing the movie"s final revenue to its budget.

- **Condition**: Ensure that both `movie_budget` and `final_movie_revenue` are non-zero and non-null to avoid division errors and ensure data integrity


In [ ]:
final_df = evaluation_utils.calculate_roi(final_df, "Movie_budget", "Final_movie_revenue")
print("There is now " + str(final_df["Final_movie_revenue"].size) + " movies to work with in the financial success analysis")

#### Data Normalization and Scaling

Given the varied scales of ratings and ROI, we standardize these features to a common scale using both Standard and Min-Max scalers. 

In [43]:
# Initialize scalers
final_df = evaluation_utils.scale_features(final_df)

#### Weighted Success Metric

To evaluate overall movie success, we calculate a weighted sum of normalized ratings and ROI, assigning double the weight to ROI given its financial significance in assessing success.

In [ ]:
# Calculate movie success which is a weighted sum choosing that the ROI feature will have double the importance of ratings
final_df = evaluation_utils.calculate_weighted_success(final_df, 0.67)

final_df.head()

### 3. Gender and age vs success

Now that we have have established our success metric, we can evaluate the effect of actors' gender on the success of a movie. 

The best way we chose in order to conduct our analysis is a **Regression analysis**. Indeed, this will allow us to quantify and understand the relationship between gender, age, and other factors on ROI, ratings and our success metric while controlling for potential confounders.

Before diving into regression, we should perform an **Exploratory Data Analysis (EDA)** to understand the relationships among variables and detect any anomalies.

#### 1.1 Exploratory Data Analysis (EDA)

First, let's list the variable of interest:

Categorical variable:
- *Actor_gender*: An actor is either Male or Female. 
- *isAdult*: A boolean. Returns 1 if it a +18 rated movie
- *Movie_genre*
- *Movie_countries*, 
- *Movie_languages*

Continous variable:
- *Average_female_actor_age*: Represents the average age of the female actresses. 
- *Average_male_actor_age*: Represents the average age of the male actresses.
- *Average_ratings* and *Movie_box_office_revenue*: These are the dependent variables of our studies.    



##### A quick comment about the dependent variables:
The goal of our study is to look at ratings, ROI and our succes metric. However, due to complications with finding enough data for the ROI and success metric estimation we will need to postpone the in-depth analysis to P3.

##### 1.1.1 Univariate Analysis of continuous variables

As a numerical analysis, we can show a table with the minimum, maximum, mean, standard deviation (SD) and median of each variable.

In [45]:
# Initialize object for EDA processing
eda = EDA(final_df.copy())

In [ ]:
pd.options.display.float_format = "{:,.2f}".format

# Convert columns to numeric and drop rows with missing data.
eda.dataframe[eda.numeric_columns] = eda.dataframe[eda.numeric_columns].apply(pd.to_numeric, errors="coerce")
eda.dataframe.dropna(subset=eda.numeric_columns, inplace=True)

# Summary statistics
summary_table = eda.summary()

summary_table

In [ ]:
eda.plot_histograms_combined(
    variables=["Average_ratings", "Final_movie_revenue", "ROI", "Movie_success"],
    title="Figure 1: Histogram of Dependent Variables",
    bins=15,
    layout=(2, 2)
)

As we can see in Figure 1, the histogram for *Average_ratings* and *Movie_success* is bell-shaped, indicating a **normal distribution** of movie ratings and success.

As for the distribution of *Final_movie_revenue* and *ROI* are highly skewed. In order to adresse that, we have to apply a log transformation to them.

In [ ]:
eda.dataframe = evaluation_utils.log_transform(eda.dataframe, ["Final_movie_revenue", "ROI"])
eda.plot_log_transformed_histograms()

Now we have successfully obtained a normal distribution for the *Final_Movie_revenue* and *ROI*. 

Let's now look at our independent variables.

In [ ]:
eda.plot_histograms_combined(
    variables=["Num_votes", "Movie_release_date", "Movie_runtime", "Female_actors", "Male_actors"],
    title="Figure 2: Histogram of Independent Variables",
    bins=15,
    layout=(2, 3)
)

The histogram for the *Movie_release_date* shows an increase in movie releases after the 1980s, with a sharp spike around the 2000s. This suggests that the number of movie releases accelerated significantly in the late 20th and early 21st centuries.

For the rest of the variables, we need to apply log transformation since they are skewed, being: *Num_votes*, *Movie_runtime*, *Female_actors* and *Male_actors*.

In [ ]:
# Apply log transformation to skewed independent variables
eda.dataframe = evaluation_utils.log_transform(eda.dataframe, ["Num_votes", "Movie_runtime", "Female_actors", "Male_actors"])

eda.plot_log_transformed_independent_histograms()

Log of Num_votes: A bimodal distribution, with most movies receiving moderate votes and a smaller group of popular movies getting significantly more votes.

Log of Movie_runtime: Right-skewed, with most movies having runtimes between 50 to 150 minutes (from Figure 2), and few with very long runtimes.

Log of Female_actors: Slightly bimodal, with most movies having a small number of female actors, and a few with more.

Log of Male_actors: Similar to female actors, the distribution is skewed towards movies with fewer male actors, and only a few movies have a large number of male actors.

In [ ]:
eda.numeric_columns = ["Average_ratings",
                   "log_Num_votes",
                   "Movie_release_date",
                   "log_Final_movie_revenue",
                   "log_ROI",
                   "log_Movie_runtime",
                   "log_Female_actors",
                   "log_Male_actors",
                   "Movie_success"]

eda.plot_boxplots()

We now have to focus on outliers and removing them for a more accurate modeling. From the boxplots, we chose the remove the outliers of the *Movie_box_office_revenue* and the *Minutes_runtime* variables. To do that, we use the **InterQuartile Range (IQR) method**, which identifies values that are significantly higher or lower than the majority of the data.

The *remove_outliers* function will calculate the IQR for each column specified and removes any rows with values outside the range:

$[Q1 - 1.5 \times \text{IQR}, Q3 + 1.5 \times \text{IQR}]$


In [ ]:
eda.numeric_columns = ["Average_ratings",
                   "log_Num_votes",
                   "Movie_release_date",
                   #"log_Final_movie_revenue",
                   "log_ROI",
                   "log_Movie_runtime",
                   "log_Female_actors",
                   "log_Male_actors",
                   "Movie_success"]

# Dataframe without outliers
eda.dataframe = data_utils.remove_outliers(eda.dataframe, eda.numeric_columns)

eda.plot_boxplots()

In [ ]:
print(f"After removing the outliers and dropping the null values, we are left with {len(eda.dataframe):,} movies.")

##### 1.1.2 Bivariate Analysis of continuous variables

In [ ]:
eda.plot_correlation_matrix()

The correlation matrix for the continuous variables in our dataset shows the following relationships:

•⁠  *log_⁠Female_actors* and *log_Male_actors*: The correlation coefficient is *0.11*, indicating a weak positive correlation. This suggests that movies with a higher number of female actors also tend to have a higher number of male actors. We note that this could indicate a general tendency for larger casts to include more actors of both genders.

•⁠  *log_⁠Final_movie_revenue* and *log_Male_actors*: The correlation coefficient is *0.27*, indicating a moderate positive correlation. This suggests that movies with more male actors might achieve higher box office earnings. 

•⁠  *log_Female_actors* and *Movie_release_date*: The correlation coefficient is *0.15*, indicating a weak positive correlation. This suggests that more recent movies tend to include a higher number of female actors. 

•⁠  *Movie_success* has a strong positive correlation with *log_ROI* (0.79) and A*verage_ratings* (0.72), which makes sense since it was calculated from them.

While gender composition affects the correlation, these findings highlight the need for deeper analysis (e.g., regression) to control for potential confounders and isolate the impact of gender on box office success.

#### 1.1.3 Analysis and encoding of categorical variables
Regression analysis with many categorical variables can quickly get complicated. Indeed using a boolean value for each category except one can cause us to have too many regressors. For variables like isAdult and Gender it is easy as there are only two possibilities. However, for Movie_genre, Movie_countries and Movie_languages we can quickly get into the hundreds. Here is how we plan on managing this:

##### 1.1.3.1 Analysis of the Movies' genres

We set a cutoff threshold of 1000. This means all genres with less than 1000 occurences will be discarded. 

In [ ]:
filtered_genre_counts = eda.filter_and_count(column_name="Movie_genres", threshold=1000)

Let us check if this restricts our study to too few movies.

In [ ]:
popular_genres = eda.analyze_popular_genres(filtered_genre_counts=filtered_genre_counts)

We can drop the movies that have none of the most popular genres. We can also assume that the genres are ordered by relevance. This assumption was made after manually checking for 50 randomly sampled movies. It was true for all 50. We will thus keep for each movie, the first popular genre that appears.

In [ ]:
# Creating a new column named "Main_genre" that stores, for each movie, the first occurence of one of the popular genres
eda.get_main_genres(popular_genres=popular_genres)

We can now create the dummy variables for the "Main_genre" column using one-hot encoding.

In [ ]:
eda.dataframe = pd.get_dummies(eda.dataframe, columns=["Main_genre"], drop_first=True) # Reference category is action/adventure
eda.dataframe.sample(5)

##### 1.1.3.2 Analysis of the Movies' languages

We will plot the top 5 languages by number of occurences:

In [ ]:
eda.analyze_languages()

We have an unbalanced dataset with English being much more prominent than other languages.

Categorizing movies into two groups: 
- those available exclusively in English, 
- those available in other languages (which may include English alongside another language).

This will be done through the creation of the boolean "Is_not_only_english" which is True when the movie is available in another language.

In [ ]:
eda.categorize_languages()

##### 1.1.3.2 Analysis of the Movies' countries

In [ ]:
eda.analyze_countries()

We have a very similar situation to the languages. The United States of America have a much more prominent number of movies. We will thus proceed like previously by categorizing the movies into two groups:
- USA movies 
- Movies from other countries 

This will be done through the creation of the boolean "Is_USA_movie" which is True when the movie is from the USA.

In [ ]:
count_usa_movies = (eda.dataframe["Movie_countries"] == "united states of america").sum()
count_other_countries_movie = len(eda.dataframe) - count_usa_movies 
print(f"There are {count_usa_movies} USA movies and {count_other_countries_movie} movies from other countries.")

In [ ]:
eda.categorize_countries()

##### 1.1.3.2 Analysis of the Movies' production companies

In [ ]:
filtered_production_count = eda.analyze_production_companies()

In [65]:
prod_companies = pd.read_csv("data/Extra CSV/production_companies.csv")

box_office = eda.merge_production_data(filtered_production_count, prod_companies)


We inspect the company names and we saw that there are Columbia Pictures and Columbia Pictures Corporation. There are two names for the same company. 
We apply unify_columbia_revenue to attribute the same revenue for them. 

In [66]:
eda.unify_columbia_revenue(box_office_df=box_office)


In [ ]:
box_office["Total Worldwide Box Office"] = box_office["Total Worldwide Box Office"].replace(
    {"\$": "", ",": ""}, regex=True).astype(float)

def calculate_box_office(companies_list):
   if not companies_list:
        return 0  
   return box_office.loc[box_office["Company Name"].isin(companies_list), "Total Worldwide Box Office"].sum()

eda.dataframe["Box_office_companies"] = eda.dataframe["Production_companies_cleaned"].apply(calculate_box_office)
eda.filter_movies_with_box_office(box_office_column="Box_office_companies")

## je regle ca asap juste la ca me pete les couilles 

In [ ]:
print(eda.dataframe.columns)


### 1.2 Identifying Potential Confounders
In order to not have bias in our study, it is essential we add the confounders. This means that we have to identify the variables that have an impact on both treatment and outcome, here being the actor gender and the final box office revenue respectivly. The confounders are: 
-  ⁠Movie_genre
-  ⁠Movie_countries
- ⁠Movie_languages
- Num_votes

### 1.3 Linear Regression


In [69]:
final_df_regression = eda.dataframe.copy()

In [ ]:
print(final_df_regression.columns)

In [71]:
indep_vars = ["log_Female_actors", "log_Male_actors", "log_Movie_runtime", "Is_not_only_english", 
                   "Is_USA_movie",  "Main_genre_drama",
                   "Main_genre_thriller", "Main_genre_comedy", "Main_genre_romance_film", "Start_year",
                   "Box_office_companies", "log_Num_votes"]

dep_vars = "Log_ROI"

X = final_df_regression[indep_vars]
y = final_df_regression[dep_vars]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

X_scaled_wconst = sm.add_constant(X_scaled)

In [ ]:
# Create a DataFrame to store VIF values
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns

# Calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# Display VIF values
print(vif_data)

We can see that all VIF values are far below 5 indicating multicollinearity should not be an issue.

In [ ]:
model = sm.OLS(y, X_scaled_wconst).fit()
print(model.summary())

In [ ]:
indep_vars = ["log_Female_actors", "log_Male_actors", "Movie_runtime", "Is_not_only_english", 
                   "Is_USA_movie",  "Main_genre_drama",
                   "Main_genre_thriller", "Main_genre_comedy", "Main_genre_romance_film", "Start_year",
                   "Box_office_companies", "Log_ROI"]

dep_vars = "Normalized_Rating"

X = final_df_regression[indep_vars]
y = final_df_regression[dep_vars]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

X_scaled_wconst = sm.add_constant(X_scaled)

model = sm.OLS(y, X_scaled_wconst).fit()
print(model.summary())

In [ ]:
# Create a DataFrame to store VIF values
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns

# Calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# Display VIF values
print(vif_data)

We can see that all VIF values are far below 5 indicating multicollinearity should not be an issue.

### 4. How does it compare to streaming platforms?

In this section, we want to expand our study by analyzing data from various streaming platforms. We focus on the movies of Netflix, Amazon Prime, Disney+, and Hulu. To achieve this, we created a dataset by merging data from multiple sources found on Kaggle.


This dataset structure enables comprehensive analysis across various dimensions such as gender repartition in the cast, the ratings and description. Using these columns, we can investigate how streaming platform movies differ in content and ratings compared to traditional box office movies.



In [76]:
streaming_df = get_streaming_dataframe()

In [ ]:
streaming_df.head()

In [78]:
streaming_df.drop_duplicates(inplace=True)


a. <u>Numerical Analysis</u>: 

In [79]:
eda_streaming = EDA(streaming_df.copy())


In [ ]:
pd.options.display.float_format = "{:,.2f}".format

# Filtrer uniquement les colonnes qui existent dans la dataframe
existing_numeric_columns = [col for col in eda_streaming.numeric_columns if col in eda_streaming.dataframe.columns]

# Convert columns to numeric for existing columns and drop rows with missing data
eda_streaming.dataframe[existing_numeric_columns] = eda_streaming.dataframe[existing_numeric_columns].apply(pd.to_numeric, errors="coerce")
eda_streaming.dataframe.dropna(subset=existing_numeric_columns, inplace=True)

# Summary statistics
summary_table = eda_streaming.summary_bis()

summary_table


In [ ]:
eda_streaming.plot_histograms_combined(
    variables=["Num_votes", "Movie_release_date", "Movie_runtime", "Female_actors", "Male_actors"],
    title="Figure 2: Histogram of Independent Variables",
    bins=15,
    layout=(2, 3)
)

Release_year : The majority of movies in the dataset are very recent with an increase after the year 2000. This distribution seems to be logical beause streaming platform really took place recently.

Movie_runtime: The duration of most movies is between 90 and 120 minutes, which is typical for films.

Female_count and male_count: We can see that the number of men is greater than women but this difference is lower than for CMU/IMDB movies.



## PEUT ETRE CHANGER LE SCALE DE NUMVOTES

In [ ]:
# Apply log transformation to skewed independent variables
eda_streaming.dataframe = evaluation_utils.log_transform(eda_streaming.dataframe, ["Num_votes", "Movie_runtime", "Female_actors", "Male_actors"])

eda_streaming.plot_log_transformed_independent_histograms()

Log_Num_Votes : The histogram shows a near symmetric distribution of log transformed movie votes resembling a normal distribution. Most movies have log-votes between 6 and 10.

Log_Movie_runtime: Most movies have a log runtime between 4 and 5, with the distribution slightly skewed to the right.

Log_Male_actors and log_female_actors : The log of female actors is slightly lower than the log of male actors. 

In [ ]:
eda_streaming.numeric_columns = ["Average_ratings",
                   "log_Num_votes",
                   "Movie_release_date",
                   #"log_Final_movie_revenue",
                   #"log_ROI",   # je crois juste lui je le teje 
                   "log_Movie_runtime",
                   "log_Female_actors",
                   "log_Male_actors",
                   #"Movie_success"
                   ]

eda_streaming.plot_boxplots()

As we did in the previous part, we now have to focus on outliers and removing them for a more accurate modeling.

In [ ]:

eda_streaming.dataframe = data_utils.remove_outliers(eda_streaming.dataframe, eda_streaming.numeric_columns)

eda_streaming.plot_boxplots()

In [ ]:
print(f"After removing the outliers and dropping the null values, we are left with {len(eda_streaming.dataframe):,} movies.")

##### 4.1.2 Bivariate Analysis of continuous variables

In [ ]:
eda_streaming.plot_correlation_matrix()

The number of votes and gender composition (especially the number of male actors) seem to influence movie duration. It suggests that popularity and cast composition could impact movie production characteristics.
Weak correlations between movie duration and other variables indicate that duration might not strongly determine popularity based on votes.
This correlation matrix suggests further analysis like regression that could help separate and clarify the effects of gender and duration on popularity.

#### 4.1.3 Analysis and encoding of categorical variables
Regression analysis with many categorical variables can quickly get complicated. Indeed using a boolean value for each category except one can cause us to have too many regressors. For variables like isAdult and Gender it is easy as there are only two possibilities. However, for Movie_genre, Movie_countries and Movie_languages we can quickly get into the hundreds. Here is how we plan on managing this:

##### 4.1.3.1 Analysis of the Movies' genres

We set a cutoff threshold of 500. This means all genres with less than 500 occurences will be discarded. 

In [ ]:
filtered_genre_counts = eda_streaming.filter_and_count(column_name="Movie_genres", threshold=500)

Let us check if this restricts our study to too few movies.

In [ ]:
popular_genres = eda_streaming.analyze_popular_genres(filtered_genre_counts=filtered_genre_counts)

We choose to only keep only the movies xhich have at more 2 genres and create dummies with all other combinations.

In [89]:
eda_streaming.dataframe = eda_streaming.dataframe[eda_streaming.dataframe["Movie_genres"].apply(lambda x: len(x.split(",")) <= 2)]

In [ ]:
eda_streaming.dataframe["Movie_genres"].value_counts()

In [ ]:

dummies = pd.get_dummies(eda_streaming.dataframe["Movie_genres"], prefix="Movie_genres", drop_first=True)

dummy_columns = dummies.columns.tolist()
print("Dummies columns :", dummy_columns)


In [ ]:
#eda_streaming.dataframe = pd.get_dummies(eda_streaming.dataframe, columns=["Movie_genres"], drop_first=True) 
eda_streaming.dataframe = pd.concat([eda_streaming.dataframe, dummies], axis=1)

eda_streaming.dataframe.sample(5)

##### 1.1.3.2 Analysis of the Movies' languages

We will plot the top 5 languages by number of occurences:

In [ ]:
eda_streaming.analyze_languages()

Categorizing movies into two groups: 
- those available exclusively in English, 
- those available in other languages (which may include English alongside another language).

This will be done through the creation of the boolean "Is_not_only_english" which is True when the movie is available in another language.

In [ ]:
eda_streaming.categorize_languages()

##### 1.1.3.2 Analysis of the Movies' countries

In [ ]:
eda_streaming.analyze_countries()

We have a very similar situation to the languages. The United States of America have a much more prominent number of movies. We will thus proceed like previously by categorizing the movies into two groups:
- USA movies 
- Movies from other countries 

This will be done through the creation of the boolean "Is_USA_movie" which is True when the movie is from the USA.

In [ ]:
count_usa_movies = (eda_streaming.dataframe["Movie_countries"] == "United States").sum()
count_other_countries_movie = len(eda_streaming.dataframe) - count_usa_movies 
print(f"There are {count_usa_movies} USA movies and {count_other_countries_movie} movies from other countries.")

In [ ]:
eda_streaming.categorize_countries_bis()

##### 1.1.3.2 Analysis of the Movies' streaming platforms 

We created dummy variables for the streaming platforms(Netflix, Disney, Amazon prime). Each platform is now a column with 0 or 1, where 1 means the movie is available on that platform.

In [ ]:
eda_streaming.dataframe = pd.get_dummies(eda_streaming.dataframe, columns=["Platform"], drop_first=True) 
eda_streaming.dataframe.sample(5)

### 1.3 Linear Regression

In [99]:
streaming_df_regression = eda_streaming.dataframe.copy()

In [ ]:
print(streaming_df_regression.columns)

In [101]:
indep_vars1=["log_Female_actors", "log_Male_actors", "Num_votes", "log_Movie_runtime", "Is_not_only_english", "Is_USA_movie",
          'Platform_Disney', 'Platform_Netflix', "Is_Adult"]

indep_vars=np.hstack([indep_vars1, dummy_columns])
dep_vars = "Average_ratings"

X = streaming_df_regression[indep_vars]
y = streaming_df_regression[dep_vars]

scaler = StandardScaler()
X_scaled_streaming = scaler.fit_transform(X)

X_scaled_streaming = pd.DataFrame(X_scaled_streaming, columns=X.columns, index=X.index)

X_scaled_wconst_streaming = sm.add_constant(X_scaled_streaming)

In [ ]:
# Create a DataFrame to store VIF values
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled_streaming.columns

# Calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X_scaled_streaming.values, i) for i in range(X_scaled_streaming.shape[1])]

# Display VIF values
print(vif_data)

In [ ]:
model_streaming = sm.OLS(y, X_scaled_wconst_streaming).fit()
print(model_streaming.summary())

### 5. <u>What are the social reasons behind the presence of female characters in movies? Is it due to sexualization or genuine equality of representation?</u>

The first part of our analysis uses plot summaries to examine whether female characters are portrayed as having agency (taking action and driving the story) or are reduced to sexualized or objectified roles. So far, we have used LLMs, such as ChatGPT Plus, to classify plot summaries based on these criteria (contains/does not contain sexualization of females), which provided useful initial results and validated the approach. However, due to time constraints during milestone 2, we were unable to use more advanced methods, such as zero-shot or few-shot learning with models like OpenAI's GPT-4, T5, or BERT variants, which we plan to explore further in milestone 3.
The second part of our analysis focuses on adult-rated films in the resulting merged dataframe, specifically examining the gender of the actors to understand patterns of representation.

In [ ]:
bechdel_df = pd.read_csv("data/Bechdel test/bechdel_test_movies.csv", names=["Id", "Year", "Title", "Bechdel_rating", "tconst"], header=None)

bechdel_df = bechdel_df.drop(index=0)
bechdel_df.head()

In [ ]:
bechdel_df["tconst"] = bechdel_df["tconst"].apply(lambda x : "tt"+str(x))
bechdel_df.head()

In [ ]:
final_final_df = pd.merge(eda.dataframe.copy(), bechdel_df, on="tconst")
len(final_final_df)

In [ ]:
final_final_df.drop(columns=["tconst"])
final_final_df["Bechdel_rating"].value_counts()

In [108]:
final_final_df["Bechdel_rating"] = pd.to_numeric(final_final_df["Bechdel_rating"])

In [109]:
indep_cols = ["log_Female_actors", "log_Male_actors", "log_Movie_runtime", "Is_not_only_english", 
                   "Is_USA_movie", "Main_genre_drama",
                   "Main_genre_thriller", "Main_genre_comedy", "Main_genre_romance_film", "Start_year",
                   "Box_office_companies", "Bechdel_rating", "log_Num_votes"]

X = final_final_df[indep_cols]

y = final_final_df["Log_ROI"]

In [ ]:
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

X_scaled_wconst = sm.add_constant(X_scaled)

model = sm.OLS(y, X_scaled_wconst).fit()
print(model.summary())

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Create a DataFrame to store VIF values
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns

# Calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# Display VIF values
print(vif_data)

In [112]:
indep_cols = ["log_Female_actors", "log_Male_actors", "Movie_runtime", "Is_not_only_english", 
                   "Is_USA_movie", "Main_genre_drama",
                   "Main_genre_thriller", "Main_genre_comedy", "Main_genre_romance_film", "Start_year",
                   "Box_office_companies", "Bechdel_rating", "Log_ROI"]

X = final_final_df[indep_cols]

y = final_final_df["Normalized_Rating"]

In [ ]:
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

X_scaled_wconst = sm.add_constant(X_scaled)

model = sm.OLS(y, X_scaled_wconst).fit()
print(model.summary())

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Create a DataFrame to store VIF values
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns

# Calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# Display VIF values
print(vif_data)

In [115]:
indep_cols = ["log_Female_actors", "log_Male_actors", "Movie_runtime", "Is_not_only_english", 
                   "Is_USA_movie", "Main_genre_drama",
                   "Main_genre_thriller", "Main_genre_comedy", "Main_genre_romance_film", "Start_year",
                   "Box_office_companies", "Log_ROI"]

X = final_final_df[indep_cols]

y = final_final_df["Bechdel_rating"]

In [ ]:
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

X_scaled_wconst = sm.add_constant(X_scaled)

model = sm.OLS(y, X_scaled_wconst).fit()
print(model.summary())

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Create a DataFrame to store VIF values
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns

# Calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# Display VIF values
print(vif_data)

## New Dialogue dataset

In [ ]:
movie_dialogue_df = pd.read_csv("data/Dialogue/meta_data7.csv",names=["Script_id", "tconst", "Title", "Year", "Gross", "Lines_Data"], header=0, encoding="ISO-8859-1")
movie_dialogue_df.drop(columns=["Gross", "Lines_Data"], inplace=True)
movie_dialogue_df.head()

In [ ]:
character_dialogue_df = pd.read_csv("data/Dialogue/character_list5.csv",names=["Script_id", "Character_name", "Words", "Gender", "Age"], header=0, encoding="ISO-8859-1")
character_dialogue_df.dropna(subset=["Character_name", "Age"], inplace=True)
character_dialogue_df.head()

In [ ]:
dialogue_df = pd.merge(movie_dialogue_df, character_dialogue_df, on="Script_id")
dialogue_df.drop(columns=["Script_id"], inplace = True)
dialogue_df.head()

In [ ]:
final_dialogue_df = dialogue_df.groupby("tconst").agg(
    Characters=("Character_name", list),
    Words=("Words", list),
    Gender=("Gender", list)
)
final_dialogue_df.head()

In [ ]:
dialogue_merged_df = pd.merge(final_df, final_dialogue_df, on="tconst")
len(dialogue_merged_df)